In [1]:
import pickle
import requests

In [40]:
# from https://cloud.gate.ac.uk/shopfront/displayItem/tagger-pos-pl-maxent1 - "A POS tagger for pl / Polish using the Universal 
# Dependencies POS tagset.
# This tagger is based on a simple maximum entropy model trained on the corpus from the universal dependencies collection using 
# the GATE Learning Framework plugin.
# The model is trained on all available corpora, except the test corpus. Evaluation on the test set gives 0.9456 accuracy. 
# Accuracy on out-of-vocabulary words (words not seen in the trainin set) is 0.9122 (case-sensitive) / 0.9174 (not 
# case-sensitive)."

URL = 'https://cloud-api.gate.ac.uk/process/tagger-pos-pl-maxent1'
PARAMS = {
    'Content-Type': 'text/plain',
    'Accept': 'application/json'
}
file = 'memoirs.txt'

In [33]:
def retrieve_text(filename, split_lines=True):
    with open(filename) as f:
        lines = f.readlines()
        
    if split_lines==True:
        return lines
    else:
        full_text = []
        for line in lines:
            line = line.strip('\n')
            full_text.append(line)
        full_text = " ".join(full_text)
        return full_text

In [68]:
text = retrieve_text(file)[1]
print(text)

Antonich było dwóch brat Jana a stryj brata Melchiora i Antoni brat Melchiora. Niewiem o ile to prawda – ale mi mówił kuzyn nasz Karol Czermiński – jakoby Dziad jego a stryj Melchiora zaopiekował się młodzieniaszkiem i wziął go do siebie na wychowanie po śmierci Rodziców. Był podobno Starostą , ale jakim niepamiętam . Był w zażyłych stosunkach z Domem Potockich, prawdopodobnie, Dziedziców Brzeżan i Skolego w Stryjskim. Gdy zaś Pan Potocki odwiedzał Pana Starostę, znając dobrze całą jego rodzinę – spostrzegł młodego Melchiora – ładny to był chłopczyna więc zapytał Pan Potocki co to za jeden. Odpowiedział mu Pan Starosta całą rzecz – a wtenczas Pan Potocki zaproponował, że chce go wziąć na swój dwór. I stało się, że Melchiorek sprytny i szykowny z natury rychło ujął sobie Pana Potockiego, i tak umiał sobie zjednać zaufanie swojego Patrona, że ten jadąc raz do dóbr swoich Brzeżany i Skolego wziął go sobie za sekretarza. Przyjechawszy do Skolego przekonał się, że te rozległe dobra w górach

In [69]:
# needs testing
class TaggerAnnotations:
    def __init__(self, url, headers, data):
        r = requests.post(url=url, data=data.encode('utf-8'), headers=headers) 
        self.annotations = r.json()
        
        tagger_annotations = []
        for entry in self.annotations['entities']['Token']:
            word = entry['string']
            pos = entry['upos']
            confidence = entry['LF_confidence']
            target_list = entry['LF_target_list']
            confidence_list = entry['LF_confidence_list']

            annotation = [word, pos, confidence, target_list, confidence_list]
            tagger_annotations.append(annotation)
        self.processed_annotations = tagger_annotations
        
    def print_annotations(self):
        for entry in self.processed_annotations:
            print(entry[0] + ' | ' + entry[1] + ' | ' + str(entry[2]))
        

In [70]:
tagger_anns = TaggerAnnotations(URL, PARAMS, text)

In [71]:
tagger_anns.print_annotations()

Antonich | ADJ | 0.6547036018399806
było | VERB | 0.7934050203007446
dwóch | NUM | 0.8510977315356847
brat | NOUN | 0.9991358370469045
Jana | PROPN | 0.9921619163091637
a | CCONJ | 0.9510893519738102
stryj | NOUN | 0.7882762270277356
brata | NOUN | 0.9958829492483253
Melchiora | PROPN | 0.9927259793317229
i | CCONJ | 0.9927751253090022
Antoni | PROPN | 0.999086427666739
brat | NOUN | 0.9259167533478992
Melchiora | PROPN | 0.9956417382620774
. | PUNCT | 0.9999999850526142
Niewiem | NOUN | 0.4203488332319943
o | ADP | 0.9678288676981368
ile | PRON | 0.6653671837859164
to | VERB | 0.8830547630740087
prawda | NOUN | 0.9413877427964505
– | PUNCT | 0.9999025452133056
ale | CCONJ | 0.9346392269497616
mi | PRON | 0.911182111732338
mówił | VERB | 0.977504184873082
kuzyn | NOUN | 0.5897727107218459
nasz | DET | 0.8271679935022845
Karol | PROPN | 0.9906137946437641
Czermiński | PROPN | 0.9958918584445882
– | PUNCT | 0.9993481453680437
jakoby | SCONJ | 0.6491219731442415
Dziad | PROPN | 0.86238038

In [64]:
tagger_anns.annotations

{'text': 'Tam się ożenił z Panną Kobylską, zacnego rodu osobą i miał z nią trzech synów Michała, Józefa i Kazimierza – niebawem owdowiał. Około roku 1784 pojechał śp. Melchior za wołami. W drodze na Posadzie Chyrowskiey zapędziły się woły w zboże i parę sztuk zajęto do dworu. Właścicielką była z domu Dzianott wdowa po dwóch mężach – z pierwszym Pierściorowskim miała córkę Maryę, Kleofasę. Ustroiwszy się pokaźnie pojechał do dworu by zapłacić za szkodę – ale o wynagrodzeniu nie było mowy. Zastawiono sute śniadanie z starym winem – rada była gosposia gościowi a śp. Melchior nie namyślając się, gdy mu wdówka wpadła w oko – bez wielu korowodów odezwał się do niej: - „wiesz Asińdźka co? Jesteś wdowa – ja wdowiec – pobierzmy się.” A na to Pani Joanna – „Ja już z zamęścia zrezygnowałam, ale że mi się Waszmość podobałeś, więc jeżeli wola Twoja oddam Ci moją Kleosię”. W alkierzyku słyszała Kleosia całą rozmowę – można sobie wyobrazić co działo się z biedną dziewczyną – gdy matka zawołała ją do 

In [49]:
tagger_anns.processed_annotations

[['Dziad',
  'NOUN',
  0.7880786605621269,
  ['NOUN',
   'PROPN',
   'VERB',
   'PART',
   'ADJ',
   'ADV',
   'PRON',
   'ADP',
   'X',
   'AUX',
   'NUM',
   'CCONJ',
   'DET',
   'SCONJ',
   'PUNCT'],
  [0.7880786605621269,
   0.10489510292170405,
   0.027720336902426913,
   0.018035291525443625,
   0.017980053962304694,
   0.013517882831340243,
   0.01216509694709733,
   0.0060562273826352195,
   0.004075824690180677,
   0.0022669842614183424,
   0.001946130303838891,
   0.0013286172956452696,
   0.0010223203841544658,
   0.000901971103838763,
   9.49892584455281e-06]],
 ['mój',
  'DET',
  0.2910872818352366,
  ['DET',
   'VERB',
   'AUX',
   'CCONJ',
   'NOUN',
   'PART',
   'ADJ',
   'ADP',
   'ADV',
   'PRON',
   'SCONJ',
   'X',
   'NUM',
   'PROPN',
   'PUNCT'],
  [0.2910872818352366,
   0.27297554868377827,
   0.21909272484437967,
   0.053556895383268466,
   0.05203179616442586,
   0.04100637533155515,
   0.02861265548432886,
   0.015236516204534414,
   0.009614096521121718,


In [73]:
r = requests.post(url=URL, data='wzięłam'.encode('utf-8'), headers=PARAMS) 
print(r.json())

{'text': 'wzięłam', 'entities': {'Token': [{'indices': [0, 7], 'string': 'wzięłam', 'length': '7', 'LF_confidence_list': [0.9734626897901111, 0.008779012372705312, 0.0036861577641024473, 0.0034215420769221094, 0.002850574225594114, 0.002741946319860353, 0.002253565124627218, 0.0010870422947912798, 0.0009393171404174608, 0.0006159356302999772, 8.806809463999731e-05, 5.2149427100591925e-05, 2.134525063825404e-05, 5.699282864992934e-07, 8.455990329030539e-08], 'orth': 'lowercase', 'kind': 'word', 'suf2': 'am', 'suf3': 'łam', 'gate.LF.target': 'VERB', 'suf4': 'ęłam', 'wiki-knn-100': '38', 'minorType': 'mb-kmeans.100', 'upos': 'VERB', 'LF_target': 'VERB', 'suf5': 'ięłam', 'wordShapeShort': 'a', 'LF_target_list': ['VERB', 'NOUN', 'PART', 'ADV', 'CCONJ', 'AUX', 'PRON', 'DET', 'SCONJ', 'ADP', 'NUM', 'ADJ', 'X', 'PUNCT', 'PROPN'], 'majorType': 'wiki.pl', 'LF_confidence': 0.9734626897901111, 'language': 'pl'}]}}
